In [21]:
# 因果推断
# 竞争站点价差、或价格，或价格变化是否对我方整体销量/石油/柴油构成影响


In [22]:
### STEP1：读取数据 ###

import pandas as pd
from IPython.display import Image, display

COLS_DATE = 'date'
COLS_PRICE = '60000359_price'                   #92号国六B
COLS_PRICE_OTHER = '60000359_price_other'


COLS_X= '进站加油车量（柴油）'
COLS_Y = '60000359_amount'  

# COLS_CAR_FLOW = '进站加油车量（汽油）'
COLS_ALL = [COLS_DATE, 
            COLS_X,
            COLS_PRICE, COLS_PRICE_OTHER, COLS_Y
            ]


# 读取excel文件并获取指定列
df = pd.read_csv('../Data/竞争站分析-竞争站-单站-32801817.csv', usecols=COLS_ALL, encoding='GBK')
print(df.head())


       date  进站加油车量（柴油）  60000359_price 60000359_price_other  60000359_amount
0  2023/1/1          12            7.20         [5.07, 7.20]        186516.28
1  2023/1/2          22            7.20         [5.07, 7.20]        247329.50
2  2023/1/3          13            7.20         [5.07, 7.20]        314626.46
3  2023/1/4           7            7.41   [5.07, 7.21, 7.41]         18120.33
4  2023/1/5          19            7.41         [5.07, 7.41]         98104.99


In [23]:
### STEP2：特征处理 ###
df = df.drop(columns=[COLS_DATE], axis=1)
print(df)

     进站加油车量（柴油）  60000359_price 60000359_price_other  60000359_amount
0            12            7.20         [5.07, 7.20]        186516.28
1            22            7.20         [5.07, 7.20]        247329.50
2            13            7.20         [5.07, 7.20]        314626.46
3             7            7.41   [5.07, 7.21, 7.41]         18120.33
4            19            7.41         [5.07, 7.41]         98104.99
..          ...             ...                  ...              ...
360          10            7.16   [5.07, 7.01, 7.16]         27688.39
361          17            7.16   [5.07, 7.01, 7.16]         57224.09
362           9            7.16   [5.07, 7.01, 7.16]         37587.56
363          13            7.16   [5.07, 7.01, 7.16]        187284.47
364          13            7.16   [5.07, 7.01, 7.16]         25848.79

[365 rows x 4 columns]


In [28]:
from dowhy import CausalModel
import dowhy.datasets

# Load some sample data
data = dowhy.datasets.linear_dataset(
    beta=10,
    num_common_causes=5,
    num_instruments=2,
    num_samples=10000,
    treatment_is_binary=True)

# I. Create a causal model from the data and given graph.
model = CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"])  # Or alternatively, as nx.DiGraph

# II. Identify causal effect and return target estimands
identified_estimand = model.identify_effect()
print(identified_estimand)

# III. Estimate the target estimand using a statistical method.
estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.propensity_score_matching")

# IV. Refute the obtained estimate using multiple robustness checks.
refute_results = model.refute_estimate(identified_estimand, estimate,
                                       method_name="random_common_cause")

Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                       
─────(E[y|W3,W2,W1,W0,W4])
d[v₀]                     
Estimand assumption 1, Unconfoundedness: If U→{v0} and U→y then P(y|v0,W3,W2,W1,W0,W4,U) = P(y|v0,W3,W2,W1,W0,W4)

### Estimand : 2
Estimand name: iv
Estimand expression:
 ⎡                              -1⎤
 ⎢    d        ⎛    d          ⎞  ⎥
E⎢─────────(y)⋅⎜─────────([v₀])⎟  ⎥
 ⎣d[Z₀  Z₁]    ⎝d[Z₀  Z₁]      ⎠  ⎦
Estimand assumption 1, As-if-random: If U→→y then ¬(U →→{Z0,Z1})
Estimand assumption 2, Exclusion: If we remove {Z0,Z1}→{v0}, then ¬({Z0,Z1}→y)

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!

